In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('C:/Users/Iasonas/PycharmProjects/EntityMatching_Thesis/DataFusion/tutorials')

import fuse
fuse_env = fuse.Fuse(verbose=True)
fuse_session = fuse_env.get_session('test')

# MATCHER='DEDUPE'
MATCHER='AHC'

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Iasonas\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using versi

In [2]:
if MATCHER=='DEDUPE':
    from matching.dedupe import DedupeMatcher
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                                  'fusion_test', matching_strategy=DedupeMatcher)

if MATCHER=='AHC':
    from matching.collective_matching_ahc import AgglomerativeHierarchicalClustering
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                               'fusion_test', matching_strategy=AgglomerativeHierarchicalClustering)


In [3]:
fuse_session.train_matchers('../../datasets/current/matched.json')

In [33]:
# x=dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','doctoral_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','Bsc')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Msc','Bsc')
x=dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].cluster_simrank_buffer
{k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('manager','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('coder','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('doctor','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('clinician','doctor')

{frozenset({10, frozenset({frozenset({16, 18})})}): 1,
 frozenset({10, frozenset({frozenset({3, 4, 5, 8, 10, 12, 13})})}): 1,
 frozenset({10, frozenset({frozenset({7, 19})})}): 1,
 frozenset({10, frozenset({frozenset({0, 2, 20})})}): 1,
 frozenset({10, frozenset({frozenset({15})})}): 1,
 frozenset({10, frozenset({frozenset({9})})}): 1,
 frozenset({10, frozenset({frozenset({1})})}): 1,
 frozenset({10, frozenset({frozenset({17})})}): 1,
 frozenset({10, frozenset({frozenset({6})})}): 1,
 frozenset({10, frozenset({frozenset({14})})}): 1,
 frozenset({10, frozenset({frozenset({11})})}): 1,
 frozenset({10, frozenset({frozenset({15}), frozenset({7, 19})})}): 0.36,
 frozenset({10, frozenset({frozenset({14}), frozenset({16, 18})})}): 0.355,
 frozenset({10, frozenset({frozenset({9}), frozenset({0, 2, 20})})}): 0.347,
 frozenset({10, frozenset({frozenset({11}), frozenset({17})})}): 0.34,
 frozenset({10,
            frozenset({frozenset({11}),
                       frozenset({3, 4, 5, 8, 10, 12, 1

In [ ]:
fuse_session.match_observations()

In [ ]:
tr_clusters = fuse_session.find_true_clusters()

In [ ]:
true_facts = fuse_session.find_true_facts(persist=True, no_weights=False)
for f in true_facts:
    print(f.canonicalTuple)

In [ ]:
eval = fuse_session.evaluate(inferred_facts=true_facts,grd_path='../../datasets/current/eval.json')
print(eval)